In [1]:
import keras
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, AveragePooling2D, Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras.datasets import cifar100
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from tqdm import tqdm
import pickle
import os
import numpy as np
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import keras
import keras.backend as K
from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

import albumentations as albu
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
!pip install tensorflow==1.14.0

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [4]:
batch_size = 256
num_classes = 100
epochs = 20
img_rows, img_cols = 32, 32
lrs=[]
n = 3
depth = n * 9 + 2
input_shape = x_train.shape[1:]

In [3]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train=x_train[:10000]
y_train=y_train[:10000]
x_test=x_test[:2000]
y_test=y_test[:2000]

In [5]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [6]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [8]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [9]:
def resnet_v2(input_shape, depth, num_classes=100):

    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')

    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)

    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)


    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:
                    activation = None
                    batch_normalization = True
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0: 
                    strides = 2  


            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=True)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [10]:
model = resnet_v2(input_shape=input_shape, depth=depth)

In [11]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [12]:
def lr_schedule(epoch):
    global K1, K2

    Kz = 0.
    S = 0.
    
    sess = K.get_session()
    max_wt = 0.
    print('Computing max_wt...')
    for weight in model.weights:
        norm = np.linalg.norm(weight.eval(sess))
        if norm > max_wt:
            max_wt = norm
    print('max_wt =', max_wt)
    
    for i in tqdm(range((len(x_train) - 1) // batch_size + 1)):
        start_i = i * batch_size
        end_i = start_i + batch_size
        xb = x_train[start_i:end_i]
    	
        tmp = np.array(func([xb]))
        activ = np.linalg.norm(tmp)
        sq = np.linalg.norm(np.square(tmp))

        if sq > S:
            S = sq
        
        if activ > Kz:
            Kz = activ

    K_ = ((num_classes - 1) * Kz) / (num_classes * batch_size)
    S_ = (num_classes - 1) ** 2 / (num_classes * batch_size) ** 2 * S
    
    K1 = beta1 * K1 + (1 - beta1) * K_
    K2 = beta2 * K2 + (1 - beta2) * S_

    lr = (np.sqrt(K2) + K.epsilon()) / K1
    lrs.append(lr)
    print('Epoch', epoch+1, 'LR =', lr,' K1= ',K1,' K2= ',K2)
    return lr

In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.1, decay=0.01),
              metrics=['accuracy'])

In [14]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [18]:
func = K.function([model.layers[0].input], [model.layers[-2].output])
lrs = []
K1 = 0.
K2 = 0.
beta1 = 0.9
beta2 = 0.999

In [20]:
save_dir = '.'
model_name = 'cifar100_model.{epoch:03d}.h5'
filepath = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='accuracy',
                             verbose=1,
                             save_best_only=True,
                             period=1)

In [21]:
history=model.fit(x_train,y_train,batch_size=128,epochs=100,validation_data=(x_test,y_test),callbacks=[lr_scheduler,checkpoint])

  0%|          | 0/40 [00:00<?, ?it/s]

Train on 10000 samples, validate on 2000 samples
Epoch 1/100
Computing max_wt...
max_wt = 722.6988


100%|██████████| 40/40 [02:18<00:00,  3.45s/it]


Epoch 1 LR = 0.11261545429227048  K1=  0.42219828244758045  K2=  0.0022606174925580615
10000/10000 [==============================] - 590s 59ms/step - loss: 2.8895 - accuracy: 0.3154 - val_loss: 3.6836 - val_accuracy: 0.1985

Epoch 00001: accuracy improved from -inf to 0.31540, saving model to ./cifar100_model.001.h5


  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]


Epoch 00038: accuracy did not improve from 0.93770
Epoch 39/100
Computing max_wt...
max_wt = 485.33157


100%|██████████| 40/40 [02:22<00:00,  3.57s/it]


Epoch 39 LR = 0.10158316809889413  K1=  0.4675696780700907  K2=  0.002255975382863978
 2560/10000 [======>.......................] - ETA: 12:20 - loss: 0.7281 - accuracy: 0.9086

KeyboardInterrupt: ignored